# Выбор локации для скважины

Заказчик исследования - компания «ГлавРосГосНефть», которой необходимо определиться, где бурить новую скважину. Входные данные от компании - пробы нефти в трёх регионах, где в каждом 10 000 месторождений с измеренными качеством нефти и объёмом её запасов. Характеристики для каждой скважины в регионы уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль, а также проанализировать возможную прибыль и риски.

Результаты исследования будут использованы заказчиком в коммерческих целях.

**Цель исследования**: построить модель для определения региона с наибольшей возможной прибылью и проанилизровать эту прибыль и риски

Ход исследования:

Входные данные от компании - статистика проб нефти в трёх регионах. Будем называть их нулевым (0), первым (1) и вторым (2) регионами. Перед анализом данные необходимо загрузить и подготовить к анализу. Далее акцентируем внимание на построение качественной модели для предсказания возможной прибыли. После построения модели необходимо будет провести тестирование на тестовой выборке. В завершении исследования проведём расчёты прибыли и рисков.

Таким образом, исследование пройдёт в 4 этапа:

* 1. Изучение данных и подготовка.
* 2. Обучение и проверка моделей.
* 3. Подготовка к расчёту прибыли.
* 4. Расчёт прибыли и рисков.

Дополнительная информация от заказчика по шагам для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Изучение данных и подготовка

Перед тем, как приступить к изучению данных из файла, импортируем все нужные для текущего исследования библиотеки и модули:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from scipy import stats as st

Также сохраним в начале исследования необходимые константные переменные:

In [2]:
RANDOM_STATE = np.random.RandomState(28)

Прочитаем полученные данные и сохраним их в таблицах `df_n`:

In [3]:
df_0 = pd.read_csv('/datasets/geo_data_0.csv')
df_1 = pd.read_csv('/datasets/geo_data_1.csv')
df_2 = pd.read_csv('/datasets/geo_data_2.csv')

Выведем на экран 3 первые строки каждой таблицы:

In [4]:
display(df_0.head(3))
display(df_1.head(3))
display(df_2.head(3))

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910


Получим общую информацию о полученных таблицах:

In [5]:
df_0.info()
df_1.info()
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

В каждой таблице содержится 100 000 наблюдений и 5 признаков. 4 признака имеют тип данных float, 1 - object.  Предварительно можно утверждать, что данных для построения модели достаточно. Согласно документации к данным:

* `id` — уникальный идентификатор скважины;
* `f0`, `f1`, `f2` — три признака точек;
* `product` — объём запасов в скважине (тыс. баррелей).

Целевым признаком является `product` - объём запасов в скважине. Замена типов данных не потребуется, однако у нас есть 1 признак с типом object, который не стоит использовать при обучении модели. Не будем исключать его из таблиц, но избавимся, установив его как `id` таблиц, которым он, по сути, и является. Иная предобработка данных не понадобится - данные от компании соответствуют высокому качеству.


In [6]:
df_0 = df_0.set_index('id')
df_1 = df_1.set_index('id')
df_2 = df_2.set_index('id')

Посмотрим на корреляции в данных. Для этого применим специальный метод библиотеки `pandas`:

In [7]:
display(df_0.corr())
display(df_1.corr())
display(df_2.corr())

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


В 0 регионе видно, что признаки `f0` и `f1` имеют отрицательную корреляцию средней силы. Также во всех случаях имеется корреляция признака `f2` с `product`, при этом в 0 и 2 регионе она достигает 0.483 и 0.445 соответственно, а в 1 - практически 1. Вероятно, именно признак `f2` особо важен для скважины и прямо связан с объёмом запасов баррелей в ней.

### Вывод

В таблицах имеются данные скважин в 3 регионах, по 100 000 месторождений в каждом. По каждому из них имеется набор признаков, влияющих на прибыльность скважины. 

Можно утверждать, что выборка достаточна для построения качественной модели. В данных выявлена значимая корреляция для всех регионов: `f2` и `product` имеют корреляцию 0.483 и 0.445 в 0 и 2 регионах соответственно, а в 1 - практически 1. Можем предположить, что этот признак прямо свяжан с объемом запасов барралей в ней и, вероятно, будет наиболее важен при прогнозировании прибыльность того или иного региона. 

Так как предобработка данных не потребуется, можем переходить к обучению и проверке моделей.


## Обучение и проверка моделей

Необходимо разбить данные на 2 выборки: обучающую и валидационную. Тестирование модели пропустим. Перед этим нужно разделить наши признаки на целевой и те, которые будут влиять на целевой признак. Напишем функцию, которая это выполнит:

In [8]:
def t_t_s(data):
    features = data.drop('product', axis=1)
    target = data['product']
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                              test_size=0.25, random_state=RANDOM_STATE)
    return features_train, features_valid, target_train, target_valid

Теперь вызовем функцию для каждой из таблиц, чтобы получить выборки по каждой:

In [9]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = t_t_s(df_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = t_t_s(df_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = t_t_s(df_2)

Для обучения возьмём модель линейной регрессии, так как остальные являются достаточно непредсказуемыми. Напишем функцию, которая будет обучать модель, находить предсказания по сырью и считать RMSE для каждой модели, а также возвращать предсказания:

In [10]:
def model_func(features_train, features_valid, target_train, target_valid, n_region):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    rmse = (mean_squared_error(target_valid, predictions)) ** 0.5
    print('Средний запас предсказанного сырья в регионе', n_region, 'составляет:', predictions.mean())
    print('RMSE модели для региона', n_region, 'равно:', rmse)
    return predictions

Теперь вызовем функцию для каждого из трёх регионов и получим результаты:

In [11]:
predictions_0 = model_func(features_train_0, features_valid_0, target_train_0, target_valid_0, 0)
predictions_1 = model_func(features_train_1, features_valid_1, target_train_1, target_valid_0, 1)
predictions_2 = model_func(features_train_2, features_valid_2, target_train_2, target_valid_0, 2)

Средний запас предсказанного сырья в регионе 0 составляет: 92.43841916872472
RMSE модели для региона 0 равно: 37.70262411816397
Средний запас предсказанного сырья в регионе 1 составляет: 68.97682121011523
RMSE модели для региона 1 равно: 67.87039361977857
Средний запас предсказанного сырья в регионе 2 составляет: 95.05679346614203
RMSE модели для региона 2 равно: 48.59274081053087


### Вывод

На этом этапе для каждого региона была построена своя линейная регрессия. Данная модель позволила определить средние запасы предсказанного сырья в каждом из регионе. Таким образом, результаты следующие:

* 0 регион - 92.27
* 1 регион - 68.73
* 2 регион - 95.16

Следовательно, пока лидирует регион 2 с наиболее большими объёмами запаса нефти, однако 0 регион также отстаёт не слишком сильно. Стоит также обратить внимание на метрику `RMSE`, рассчитанную для каждой из моделей:

* 0 регион - 37.67
* 1 регион - 0.89
* 2 регион - 40.01 

По данной метрике наиболее оптимальным является 2 регион, так как 0 и 2 имеют слишком большой разброс признака. Возможно, линейная регрессия не совсем подходит для объяснения признака в данных регионах. Следовательно, решение о выборе региона будет иметь определённые риски, которые понадобится рассчитать, однако сейчас можем переходить к расчёту прибыли.

## Подготовка к расчёту прибыли

Перейдём к расчётам прибыли, однако перед этим сохраним отдельно все нужные переменные, чтобы при необходимости была возможность скорректировать их для новых расчётов:

In [12]:
NUMBER_OF_POINTS = 500 #количество точек при разведке региона
BEST_POINTS = 200 #количество точек, выбранных с учётом машинного обучения, для разработки
BUDGET = 10000000000 #бюджет на разработку
INCOME_BARREL = 450000 #цена 1000 баррелей

Теперь приступим к расчётам. Найдём безубыточный объём сырья для разработки новой скважины:

In [13]:
MIN_GROSS_PRODUCT = BUDGET / BEST_POINTS / INCOME_BARREL
print('Минимальный безубыточный объём сырья составляет:', MIN_GROSS_PRODUCT)

Минимальный безубыточный объём сырья составляет: 111.11111111111111


### Вывод

Исследование подготовлено для расчётов прибыли, даже если экономические характеристики будут скорректированы. Минимальный безубыточный объём сырья составляет 111.11, однако в регионах предположительно этих запасов меньше. Следовательно, нужно будет с точностью подбирать точки, чтобы не получить отрицательную прибыль. Перейдём к расчётам прибыли и рисков.

## Расчёт прибыли и рисков 

Напишем функцию для расчёта прибыли:

In [14]:
def revenue(target, predictions):
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[predictions_sorted.index][:BEST_POINTS]
    profit = selected.sum() * INCOME_BARREL - BUDGET 
    return profit

Посчитаем риски и прибыль по каждому региону. Применим технику `Bootstrap` с 1000 выборок, чтобы найти распределение прибыли:

In [15]:
def bootstrap_1000(target, predictions):
    values = []
    count = 0
    for i in range(1000):
        target_sample = pd.Series(target).reset_index(drop=True).sample(n=500, random_state=RANDOM_STATE, replace=True)
        prob_sample = pd.Series(predictions)[target_sample.index]
        rev_sample = revenue(target, prob_sample)
        if rev_sample < 0:
            count += 1
        values.append(rev_sample)
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return values.mean(), lower, upper, count / 10

При помощи написанной функции найдём доверительные интервалы, среднюю прибыль и риск убытков для каждого региона:

In [20]:
mean_revenue, min_conf_interval, max_conf_interval, prob_loss = bootstrap_1000(target_valid_0, predictions_0)
print('Доверительный интервал для региона 0:', '(',min_conf_interval, ',', max_conf_interval, ')')
print('Средняя прибыль для региона 0:', mean_revenue)
print('Риск убытков для региона 0:', prob_loss, '%')
print()
mean_revenue, min_conf_interval, max_conf_interval, prob_loss = bootstrap_1000(target_valid_1, predictions_1)
print('Доверительный интервал для региона 1:', '(',min_conf_interval, ',', max_conf_interval, ')')
print('Средняя прибыль для региона 1:', mean_revenue)
print('Риск убытков для региона 1:', prob_loss, '%')
print()
mean_revenue, min_conf_interval, max_conf_interval, prob_loss = bootstrap_1000(target_valid_2, predictions_2)
print('Доверительный интервал для региона 2:', '(',min_conf_interval, ',', max_conf_interval, ')')
print('Средняя прибыль для региона 2:', mean_revenue)
print('Риск убытков для региона 2:', prob_loss, '%')

Доверительный интервал для региона 0: ( -121281790.33538921 , 931490487.013949 )
Средняя прибыль для региона 0: 400380469.9463191
Риск убытков для региона 0: 6.1 %

Доверительный интервал для региона 1: ( 58031794.45250913 , 872255600.3070915 )
Средняя прибыль для региона 1: 453570050.0259676
Риск убытков для региона 1: 1.4 %

Доверительный интервал для региона 2: ( -159862162.11521953 , 894999348.7662085 )
Средняя прибыль для региона 2: 359792760.8543889
Риск убытков для региона 2: 10.0 %


### Вывод

После расчёта прибыли и рисков по регионам можно переходить к составлению выводов. Как видно, регион 1 имеет минимальный уровень риска на уровне 1.4%, при этом доверительный интервал сообщает, что с вероятностью 95% прибыль будет не менее 5.8 миллиардов, но не более 8.7. Регион 2 имеет наиболее высокий риск 10.0% и этот риск непосопоставим с возможными доходами от региона. Регион 0 имеет риск 6.1%, однако и прибыль при лучших условиях может составить до 9.3 миллиардов. При желании пойти на риск можно выбрать регион 0, однако наиболее оптимальным с точки зрения минимизации рисков и максимизации прибыли будет именно регион 1.

## Вывод

Целью исследования было построение модели для определения возможной прибыли и расчёт прибыли и рисков. По результатам анализа данных было решено использовать модель линейной регрессии, так как остальные представляются недостаточно предсказуемыми. Метрика RMSE продемонстрировала, что регион 1 имеет наиболее низкие показатели 0.89, что может привести к более предсказуемым результатам при выборе этого региона. Регион 0 имеет RMSE чуть более 37, а 2 - 40, что делает необходимость более детальных выборов точек в регионе, однако даже в таком случае нет гарантий достаточных количеств продукта в регионе.

Исходя из расчётов прибыли, было решено предложить заказчику для выбора регион 1 с рисками в 1.4% и наибольшей средней прибылью. В случае желания пойти на риск в 6.1% и добиться максимизации прибыли можно выбрать регион 0 со средней прибылью 4 миллиарда, однако в таком сулчае появляется шанс потерять достаточно много денег. Тем не менее, все расчёты позволяют выбрать заказчику желаемый регион с заданным уровнем риска. На этом исследование можно считать завершённым.